<a href="https://colab.research.google.com/github/rhmes/point-cloud-compression/blob/main/point_cloud_compression_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Point Cloud Compression Demo

This notebook demonstrates installation, core functionality, and testing for the point cloud compression project.

## Environment Setup

This section prepares the environment for the point cloud compression project:

1.  **Mount Google Drive**: Connects Colab to Google Drive for persistent storage of the repository and virtual environment.
2.  **Clone/Update Repository**: Clones the project repository from GitHub to Google Drive or updates it if it already exists.
3.  **Virtual Environment Setup**: Creates and populates a virtual environment (`venv_gpu` or `venv_cpu`) in Google Drive using a setup script, only if it doesn't exist.
4.  **Prepare Environment**: Modifies Python's `sys.path` to include the virtual environment's packages, allowing the notebook to use installed dependencies across sessions.
5.  **Verify Dependencies**: Checks if key libraries from the virtual environment can be imported successfully.

In [12]:
# Connect to Google Drive (for Colab users)
try:
    from google.colab import drive
    drive.mount('/content/drive')
    print('Google Drive mounted.')
except ImportError:
    print('Not running in Colab, skipping Google Drive mount.')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Google Drive mounted.


In [13]:
# Clone repo only if not already cloned
import os
repo_url = 'https://github.com/rhmes/point-cloud-compression.git'
parent_dir = '/content/drive/MyDrive/projects'
repo_dir = os.path.join(parent_dir, 'point-cloud-compression')
os.makedirs(parent_dir, exist_ok=True)
os.chdir(parent_dir)
if not os.path.exists(repo_dir):
  print('Cloning point-cloud-compression...')
  !git clone {repo_url}
else:
  print('Updating point-cloud-compression...')
  !cd {repo_dir} && git pull
os.chdir(repo_dir)
print(f'Current working directory: {os.getcwd()}')

Updating point-cloud-compression...
Already up to date.
Current working directory: /content/drive/MyDrive/projects/point-cloud-compression


In [15]:
# Set this variable to True to install GPU dependencies, False for CPU
use_gpu = True # @param {type:"boolean"}

# Install dependencies based on the use_gpu flag

import os

if use_gpu:
  print("Setting up GPU virtual environment...")
  venv_dir = 'venv_gpu'
  requirements_file = 'requirements_gpu.txt'
else:
  print("Setting up CPU virtual environment...")
  venv_dir = 'venv_cpu'
  requirements_file = 'requirements_cpu.txt'

# Check if the virtual environment directory already exists
full_venv_dir = os.path.join(repo_dir, venv_dir)
venv_activate = full_venv_dir+ "/bin/activate"
venv_python = full_venv_dir+ "/bin/python"
if not os.path.exists(full_venv_dir):
  print(f'No virtual environment found at {venv_dir}. Running setup script...')
  # Run the setup script
  !pip install virtualenv
  !virtualenv {full_venv_dir}
  !source {venv_activate}
  !echo "Activating virtual environment..."
  !echo "Installing dependencies from {requirements_file}..."
  !echo venv_python={venv_python}
  !{venv_python} -m pip install -r {requirements_file}

  !{venv_python} -m pip install --upgrade pip setuptools wheel
  # Addintional libararies insatll
  !{venv_python} -m pip install "git+https://github.com/facebookresearch/pytorch3d.git"
  !echo "Virtual environment setup complete."

  # Addintionl lib

else:
    print(f'Virtual environment already exists at {full_venv_dir}. Skipping setup script.')

Setting up GPU virtual environment...
Virtual environment already exists at /content/drive/MyDrive/projects/point-cloud-compression/venv_gpu. Skipping setup script.


In [21]:
import sys
import os

# Prepare the virtual environment's site-packages
venv_path = full_venv_dir

# Common site-packages paths relative to the venv directory
site_packages_paths_candidates = [
  os.path.join(venv_path, 'lib', f'python{sys.version_info.major}.{sys.version_info.minor}', 'site-packages'),
  os.path.join(venv_path, 'lib64', f'python{sys.version_info.major}.{sys.version_info.minor}', 'site-packages'),
]

site_packages_path = None
for candidate in site_packages_paths_candidates:
  if os.path.exists(candidate):
    site_packages_path = candidate
    break

if site_packages_path and site_packages_path not in sys.path:
  sys.path.insert(0, site_packages_path)
  print(f"Added {site_packages_path} to sys.path")
elif site_packages_path:
  print(f"{site_packages_path} already in sys.path")
else:
  print(f"Could not find site-packages directory in {venv_path}")

# Verify that packages from the venv can be imported (optional)
try:
  import torch
  print(f"Successfully imported torch (version: {torch.__version__})")
except ImportError:
  print("Could not import torch. Ensure virtual environment is set up correctly.")

try:
  import open3d
  print("Successfully imported open3d")
except ImportError:
  print("Could not import open3d. Ensure virtual environment is set up correctly.")

try:
  import pytorch3d
  from pytorch3d.ops.knn import _KNN, knn_gather, knn_points

  print("Successfully imported pytorch3d")
except ImportError:
  print("Could not import pytorch3d. Ensure virtual environment is set up correctly.")

/content/drive/MyDrive/projects/point-cloud-compression/venv_gpu/lib/python3.11/site-packages already in sys.path
Successfully imported torch (version: 2.3.0+cu121)
Successfully imported open3d
Successfully imported pytorch3d


## Data Preparation

This step involves downloading and preparing the datasets required for the point cloud compression project. Specifically, it downloads the pre-converted ModelNet40 and ShapeNet point cloud datasets from Google Drive if they are not already present in the `data` directory, and extracts the contents of the downloaded zip files.

In [17]:
# Download ModelNet40 and ShapeNet datasets from Google Drive (Option 1 from README)
import os
import urllib.request

def download_from_gdrive(url, output_path):
    try:
        import gdown
    except ImportError:
        os.system('pip install gdown')
        import gdown
    gdown.download(url, output_path, quiet=False)

# Install rarfile for extracting .rar files
try:
    import rarfile
except ImportError:
    print("rarfile not found, installing...")
    os.system('pip install rarfile')
    import rarfile

# ModelNet40
repo_root = os.getcwd()
modelnet_url = 'https://drive.google.com/uc?id=1Isa8seckZ9oNzstlE7VZcd6wVVx8LdMF'
# Assuming the file is actually a .rar despite the variable name
modelnet_archive = 'ModelNet40_pc_8192.rar'
if not os.path.exists(repo_root + '/data/ModelNet40_pc_01_8192p'):
    print('Downloading ModelNet40 pre-converted point clouds...')
    download_from_gdrive(modelnet_url, modelnet_archive)
    try:
        with rarfile.RarFile(modelnet_archive, 'r') as rf:
            rf.extractall('data')
        os.remove(modelnet_archive)
        print('ModelNet40 download and extraction complete.')
    except rarfile.BadRarFile as e:
        print(f"Error extracting ModelNet40: {e}")
        print("Please ensure the downloaded file is a valid .rar archive.")
    except Exception as e:
        print(f"An unexpected error occurred during ModelNet40 extraction: {e}")

else:
    print('ModelNet40 dataset already exists. Skipping download.')

# ShapeNet
shapenet_url = 'https://drive.google.com/uc?id=1OzaU01kolBpfRRD0zKESYh67Hh2s2dbD'
# Assuming the file is actually a .rar despite the variable name
shapenet_archive = 'ShapeNet_pc_2048.rar'
if not os.path.exists(repo_root + '/data/ShapeNet_pc_01_2048p'):
    print('Downloading ShapeNet pre-converted point clouds...')
    download_from_gdrive(shapenet_url, shapenet_archive)
    try:
        with rarfile.RarFile(shapenet_archive, 'r') as rf:
            rf.extractall('data')
        os.remove(shapenet_archive)
        print('ShapeNet download and extraction complete.')
    except rarfile.BadRarFile as e:
        print(f"Error extracting ShapeNet: {e}")
        print("Please ensure the downloaded file is a valid .rar archive.")
    except Exception as e:
        print(f"An unexpected error occurred during ShapeNet extraction: {e}")
else:
    print('ShapeNet dataset already exists. Skipping download.')

ModelNet40 dataset already exists. Skipping download.
ShapeNet dataset already exists. Skipping download.


## Repository Scripts Overview

This section describes the core scripts from the `point-cloud-compression` repository that are executed in this notebook to demonstrate the project's functionality:


1.  **`train.py`**: This script trains the autoencoder model on the ModelNet40 training set. It takes input point cloud file paths, an output directory for the trained model, and the patch size (K) as arguments.
2.  **`compress.py`**: This script is used to compress point cloud files. It takes input point cloud file paths, an output directory for compressed files, and a trained model path as arguments.
3.  **`decompress.py`**: This script performs the decompression of the compressed point cloud files generated by `compress.py`. It requires the directory containing compressed files, an output directory for decompressed files, and the trained model path.
4.  **`eval.py`**: This script evaluates the performance of the compression and decompression process. It compares the original point clouds with the decompressed ones using metrics such as PSNR (Peak Signal-to-Noise Ratio), Chamfer distance, and bits per point (bpp). The evaluation results are typically saved to a CSV file.
5.  **`visualize.py`**: This script is used to visualize the evaluation metrics generated by `eval.py`. It reads the results from the CSV file and generates plots or figures to help analyze the compression performance.

In [22]:
# Example core functionality: List files in the data directory

def list_data_files(data_dir='data'):
    if not os.path.exists(data_dir):
        print(f"Directory '{data_dir}' does not exist.")
        return []
    files = os.listdir(data_dir)
    print(f"Files in '{data_dir}':", files)
    return files

# Run the function
data_files = list_data_files()

Files in 'data': ['ShapeNet_pc_01_2048p', 'ModelNet40_pc_01_8192p', '.gitkeep']


In [ ]:
# Train the autoencoder model on the ModelNet40 training set
!{venv_python} train.py './data/ModelNet40_pc_01_8192p/**/train/*.ply' './model/K256' --K 256

In [ ]:
# Compress point cloud test files using the trained model
!{venv_python} compress.py './data/ModelNet40_pc_01_8192p/**/test/*.ply' './data/ModelNet40_K256_compressed' './model/K256' --K 256

In [ ]:
# Decompress the compressed point cloud files
!{venv_python} decompress.py './data/ModelNet40_K256_compressed' './data/ModelNet40_K256_decompressed' './model/K256' --K 256

In [ ]:
# Evaluate the compression results using PSNR, Chamfer distance, and bpp metrics
!{venv_python} eval.py './data/ModelNet40_pc_01_8192p/**/test/*.ply' './data/ModelNet40_K256_compressed' './data/ModelNet40_K256_decompressed' './eval/ModelNet40_K256.csv' '../geo_dist/build/pc_error'

In [ ]:
# Visualize evaluation metrics and save plots
!{venv_python} visualize.py --csv './eval/ModelNet40_K256.csv' --outdir './figure/'